# Sber Project - Generate user profiles

In [ ]:
!pip install sdv

In [36]:
import pandas as pd
import gdown
import sdv
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
from sdv.sampling import Condition
from sdmetrics.single_column import KSComplement
from sdmetrics.column_pairs import CorrelationSimilarity

## Demo

In [ ]:
from ctgan import CTGAN
from ctgan import load_demo
import pandas as pd

In [ ]:
!pip install ctgan

In [ ]:
from ctgan import CTGAN
from ctgan import load_demo

real_data = load_demo()

# Names of the columns that are discrete
discrete_columns = [
    'workclass',
    'education',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'native-country',
    'income'
]

ctgan = CTGAN(epochs=1)
ctgan.fit(real_data, discrete_columns)
synthetic_data = ctgan.sample(100)

KeyboardInterrupt: ignored

In [ ]:
real_data

In [ ]:
real_data.corr()

In [ ]:
synthetic_data

## Load VK (Pikabu https://vk.com/pikabu ) data

In [10]:
import gdown
gdown.download_folder("https://drive.google.com/drive/folders/1ZsFQ71u3l15vohInvzVSTlwp7FAVV3zB", quiet=True)

['/content/VK/likes_pikabu.csv',
 '/content/VK/posts.csv',
 '/content/VK/users.csv']

In [11]:
users_df = pd.read_csv("/content/VK/users.csv", index_col=0)
users_df

,id,bdate,sex,verified,first_name,last_name,is_closed,city.id,city.title,country.id,...,universities__education_form,universities__education_form_id,universities__education_status,universities__education_status_id,universities__faculty,universities__faculty_name,universities__graduation,universities__id,universities__name,age
0,1145,NaN,1,0.0,Natalya,Trofimova,True,2.0,Saint Petersburg,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1398,1985-11-16 00:00:00,2,0.0,Konstantin,Martyanov,True,2.0,Saint Petersburg,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0
2,2703,NaN,2,0.0,Gertsog,Telmansky,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4230,NaN,2,0.0,Alexander,Alexandrov,True,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5385,NaN,1,0.0,Tatyana,Efremova,False,1.0,Moscow,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5232,824955048,NaN,2,0.0,Saren,Mortis,False,2.0,Saint Petersburg,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5233,825672221,NaN,2,0.0,Hinata,Sky,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5234,825743292,NaN,1,0.0,Manyunya,Kotova,True,2.0,Saint Petersburg,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5235,826249917,1995-03-18 00:00:00,1,0.0,Alisa,Gromova,False,62.0,Kaluga,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,543.0,КГУ им. Циолковского (бывш. КГПУ),28.0


In [12]:
f = pd.DataFrame(users_df.isna().sum().reset_index()).rename(columns={"index":"param", 0: "nancount"}).sort_values(by="nancount", ascending=False)
f[f["nancount"]>50]

,param,nancount
61,deactivated,5234
59,livejournal,5230
74,military__until,5167
69,career__until,5163
73,military__from,5156
...,...,...
24,followers_count,1654
8,city.title,1180
7,city.id,1179
9,country.id,720


Удалять нечего, deactivated оставлю, так как есть не NaN значения

In [13]:
users_df[~users_df["faculty"].isna()]["faculty"]

5       2157473.0
6             0.0
22          212.0
23          107.0
25      2268354.0
          ...    
5225          0.0
5226          0.0
5229          0.0
5230          0.0
5235          0.0
Name: faculty, Length: 1457, dtype: float64

In [14]:
cols_to_drop = ["universities__education_form_id", "personal.religion_id", "personal.religion_id", "military__country_id", "schools__class_id", "occupation.country_id", "universities__country", "schools__country", "country.id", "military__unit_id", "career__city_id", "universities__city", "occupation.city_id", "career__group_id", "schools__city", "university", "universities__id", "city.id", "universities__chair", "faculty", "universities__faculty", "schools__id", "occupation.id"]


ids = ["universities__education_form", "career__country_id", "schools__type_str", "schools__class", "country.title", "military__unit", "personal.religion", "university_name", "universities__chair_name", "universities__faculty_name", "city.title", "faculty_name", "universities__name", "schools__name", "occupation.name", "age"]
discrete_columns = ["sex", "is_closed", "has_relations", "occupation.type", "education_form", "personal.alcohol", "personal.smoking", "personal.people_main", "relation", "personal.life_main", "personal.political", "universities__education_status_id", "schools__type", "education_status", "verified", "deactivated"]
years = ["universities__education_status", "career__until", "career__from", "military__until", "military__from", "universities__graduation", "schools__year_graduated", "graduation", "schools__year_to", "occupation.graduate_year", "schools__year_from"]
dates = ["bdate"]
counters = ["counters.articles", "counters.video_playlists", "counters.clips", "counters.albums", "counters.subscriptions", "counters.clips_likes", "counters.clips_views", "counters.gifts", "counters.groups", "counters.pages", "counters.followers", "counters.videos", "counters.posts", "counters.audios", "counters.photos", "counters.clips_followers", "followers_count"]
named_entities = ["career__company", "schools__speciality", "career__position", "home_town"]
texts = ["personal.inspired_by", "games", "activities", "music", "movies", "interests", "books", "livejournal", "twitter", "skype", "first_name", "last_name"]
combined = ["personal.langs"]

In [15]:
users_df = users_df.drop(cols_to_drop, axis=1)

In [16]:
users_df

,id,bdate,sex,verified,first_name,last_name,is_closed,city.title,country.title,occupation.name,...,schools__type_str,schools__year_from,universities__chair_name,universities__education_form,universities__education_status,universities__education_status_id,universities__faculty_name,universities__graduation,universities__name,age
0,1145,NaN,1,0.0,Natalya,Trofimova,True,Saint Petersburg,Russia,СПбГУ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1398,1985-11-16 00:00:00,2,0.0,Konstantin,Martyanov,True,Saint Petersburg,Russia,СПбГУ ИТМО,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0
2,2703,NaN,2,0.0,Gertsog,Telmansky,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4230,NaN,2,0.0,Alexander,Alexandrov,True,NaN,Russia,МТУСИ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5385,NaN,1,0.0,Tatyana,Efremova,False,Moscow,Russia,РГГУ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5232,824955048,NaN,2,0.0,Saren,Mortis,False,Saint Petersburg,Russia,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5233,825672221,NaN,2,0.0,Hinata,Sky,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5234,825743292,NaN,1,0.0,Manyunya,Kotova,True,Saint Petersburg,Russia,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5235,826249917,1995-03-18 00:00:00,1,0.0,Alisa,Gromova,False,Kaluga,Russia,КГУ им. Циолковского (бывш. КГПУ),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,КГУ им. Циолковского (бывш. КГПУ),28.0


In [17]:
col_id = users_df["id"]
users_df = users_df.drop("id", axis=1)
users_df = users_df.drop(["first_name", "last_name"], axis=1)
users_df["bdate"] = pd.to_datetime(users_df["bdate"]).dt.year
users_df=  users_df.drop("age", axis=1)
users_df = users_df.drop(texts, axis=1)
users_df["sex"] = users_df["sex"].map({1:"female", 2:"male"})
users_df

## Train CTGAN

In [29]:
users_df

,bdate,sex,verified,is_closed,city.title,country.title,occupation.name,occupation.type,occupation.graduate_year,counters.albums,...,schools__type,schools__type_str,schools__year_from,universities__chair_name,universities__education_form,universities__education_status,universities__education_status_id,universities__faculty_name,universities__graduation,universities__name
0,NaN,female,0.0,True,Saint Petersburg,Russia,СПбГУ,university,2011.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1985.0,male,0.0,True,Saint Petersburg,Russia,СПбГУ ИТМО,university,2011.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,male,0.0,True,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,male,0.0,True,NaN,Russia,МТУСИ,university,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,female,0.0,False,Moscow,Russia,РГГУ,university,NaN,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5232,NaN,male,0.0,False,Saint Petersburg,Russia,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5233,NaN,male,0.0,True,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5234,NaN,female,0.0,True,Saint Petersburg,Russia,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5235,1995.0,female,0.0,False,Kaluga,Russia,КГУ им. Циолковского (бывш. КГПУ),university,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,КГУ им. Циолковского (бывш. КГПУ)


In [30]:
users_df.columns

Index(['bdate', 'sex', 'verified', 'is_closed', 'city.title', 'country.title',
       'occupation.name', 'occupation.type', 'occupation.graduate_year',
       'counters.albums', 'counters.audios', 'counters.pages',
       'counters.subscriptions', 'counters.videos', 'counters.video_playlists',
       'counters.posts', 'followers_count', 'counters.followers',
       'counters.photos', 'counters.clips', 'counters.clips_followers',
       'counters.clips_views', 'counters.clips_likes', 'skype',
       'university_name', 'faculty_name', 'graduation', 'education_form',
       'education_status', 'home_town', 'relation', 'personal.langs',
       'personal.alcohol', 'personal.life_main', 'personal.people_main',
       'personal.smoking', 'counters.gifts', 'personal.political',
       'personal.religion', 'counters.groups', 'counters.articles',
       'livejournal', 'twitter', 'deactivated', 'has_relations',
       'career__country_id', 'career__position', 'career__company',
       'career__fr

In [31]:
users_df['occupation.type'].value_counts()

university    2222
work          1162
school          11
Name: occupation.type, dtype: int64

In [34]:
real_data = users_df.loc[:, :]

In [40]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=real_data)
for col in discrete_columns+ids:
  if (col in real_data.columns):
    metadata.columns[col]["sdtype"] = "categorical"
del metadata.columns["occupation.name"]["pii"]

In [41]:
metadata

{
    "columns": {
        "bdate": {
            "sdtype": "numerical"
        },
        "sex": {
            "sdtype": "categorical"
        },
        "verified": {
            "sdtype": "categorical"
        },
        "is_closed": {
            "sdtype": "categorical"
        },
        "city.title": {
            "sdtype": "categorical"
        },
        "country.title": {
            "sdtype": "categorical"
        },
        "occupation.name": {
            "sdtype": "categorical"
        },
        "occupation.type": {
            "sdtype": "categorical"
        },
        "occupation.graduate_year": {
            "sdtype": "numerical"
        },
        "counters.albums": {
            "sdtype": "numerical"
        },
        "counters.audios": {
            "sdtype": "numerical"
        },
        "counters.pages": {
            "sdtype": "numerical"
        },
        "counters.subscriptions": {
            "sdtype": "numerical"
        },
        "counters.videos": {
   

In [42]:
synthesizer = CTGANSynthesizer(metadata, verbose=True, epochs=100)
synthesizer.fit(real_data)

PerformanceAlert: Using the CTGANSynthesizer on this data is not recommended. To model this data, CTGAN will generate a large number of columns.

Original Column Name              Est # of Columns (CTGAN)
bdate                             11
sex                               2
verified                          2
is_closed                         2
city.title                        529
country.title                     52
occupation.name                   2149
occupation.type                   4
occupation.graduate_year          11
counters.albums                   11
counters.audios                   11
counters.pages                    11
counters.subscriptions            11
counters.videos                   11
counters.video_playlists          11
counters.posts                    11
followers_count                   11
counters.followers                11
counters.photos                   11
counters.clips                    11
counters.clips_followers          11
counters.clips_view

Gen. (0.59) | Discrim. (0.97): 100%|██████████| 100/100 [07:40<00:00,  4.61s/it]


In [ ]:
df_synth = synthesizer.sample(num_rows=1000)
df_synth.to_csv("synthiesized_users.csv")
df_synth

In [43]:
suite_guests_with_rewards = Condition(
    num_rows=100,
    column_values={'sex': "female", 'city.title': "Saint Petersburg"}
)
fake_df = synthesizer.sample_from_conditions(conditions=[suite_guests_with_rewards])
fake_df.to_csv("synthesized_users_condition_sex_city.csv")
fake_df

Sampling conditions: 100%|██████████| 100/100 [00:01<00:00, 61.26it/s]


,bdate,sex,verified,is_closed,city.title,country.title,occupation.name,occupation.type,occupation.graduate_year,counters.albums,...,schools__type,schools__type_str,schools__year_from,universities__chair_name,universities__education_form,universities__education_status,universities__education_status_id,universities__faculty_name,universities__graduation,universities__name
0,NaN,female,0.0,False,Saint Petersburg,Russia,ПГУПС Императора Александра I,work,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2005.0,female,0.0,False,Saint Petersburg,Russia,NaN,university,NaN,1.0,...,NaN,NaN,1981.0,NaN,NaN,NaN,NaN,NaN,2013.0,МСИ им. Державина
2,NaN,female,0.0,False,Saint Petersburg,Russia,УИ РЭУ им. Плеханова (бывш. РГТЭУ),university,NaN,0.0,...,NaN,NaN,NaN,NaN,Full-time,NaN,NaN,NaN,NaN,NaN
3,1990.0,female,0.0,False,Saint Petersburg,Russia,Предприятие,university,NaN,15.0,...,NaN,NaN,NaN,NaN,Full-time,Alumnus (Bachelor's),7.0,NaN,NaN,NaN
4,1990.0,female,0.0,False,Saint Petersburg,Russia,NaN,work,2012.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1990.0,female,0.0,False,Saint Petersburg,NaN,СГА,NaN,2012.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013.0,NaN
96,1990.0,female,0.0,False,Saint Petersburg,Russia,NaN,university,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,1990.0,female,0.0,False,Saint Petersburg,Russia,"ЮРИСТ, 8(920)677-97-67",university,2013.0,3.0,...,NaN,NaN,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,NaN,female,0.0,False,Saint Petersburg,Russia,СПбГУКИ,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,Full-time,Alumnus (Bachelor's),2.0,Космический факультет,NaN,СПбГУ


In [59]:
fake_df.loc[:, list(fake_df.columns[:9])+['personal.langs', "has_relations", "counters.subscriptions", "counters.followers"]].sample(100)

,bdate,sex,verified,is_closed,city.title,country.title,occupation.name,occupation.type,occupation.graduate_year,personal.langs,has_relations,counters.subscriptions,counters.followers
52,NaN,female,0.0,False,Saint Petersburg,NaN,Институт управления,university,NaN,['Русский'],1,16.0,40.0
45,1991.0,female,0.0,False,Saint Petersburg,Russia,"ТОВ ""КТ Украина""",university,2012.0,NaN,0,14.0,133.0
29,NaN,female,0.0,False,Saint Petersburg,NaN,BAGGINS COFFEE. БЭГГИНС КОФЕ. Сеть кофеен,university,2012.0,NaN,0,4.0,277.0
16,NaN,female,0.0,True,Saint Petersburg,Russia,NaN,university,NaN,['Русский'],0,3.0,381.0
13,NaN,female,0.0,False,Saint Petersburg,NaN,NaN,NaN,2012.0,['Русский'],0,5.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,NaN,female,0.0,False,Saint Petersburg,Russia,NaN,work,NaN,NaN,0,3.0,545.0
31,NaN,female,0.0,False,Saint Petersburg,Russia,МАОУ Гимназия №56,university,NaN,NaN,0,5.0,NaN
58,1990.0,female,0.0,False,Saint Petersburg,NaN,NaN,university,2012.0,NaN,0,5.0,281.0
34,2002.0,female,0.0,False,Saint Petersburg,Russia,NaN,university,NaN,NaN,0,16.0,10.0


In [50]:
fake_df.columns

Index(['bdate', 'sex', 'verified', 'is_closed', 'city.title', 'country.title',
       'occupation.name', 'occupation.type', 'occupation.graduate_year',
       'counters.albums', 'counters.audios', 'counters.pages',
       'counters.subscriptions', 'counters.videos', 'counters.video_playlists',
       'counters.posts', 'followers_count', 'counters.followers',
       'counters.photos', 'counters.clips', 'counters.clips_followers',
       'counters.clips_views', 'counters.clips_likes', 'skype',
       'university_name', 'faculty_name', 'graduation', 'education_form',
       'education_status', 'home_town', 'relation', 'personal.langs',
       'personal.alcohol', 'personal.life_main', 'personal.people_main',
       'personal.smoking', 'counters.gifts', 'personal.political',
       'personal.religion', 'counters.groups', 'counters.articles',
       'livejournal', 'twitter', 'deactivated', 'has_relations',
       'career__country_id', 'career__position', 'career__company',
       'career__fr

## Метрики качества генерации

Здесь строится Contingency матрица и считается разница вероятностей в двух матрицах

In [56]:
KSComplement.compute(
    real_data=users_df['is_closed'],
    synthetic_data=fake_df['is_closed']
)


0.834361275539431

Метрика довольно хорошая

In [57]:
KSComplement.compute(
    real_data=users_df['sex'],
    synthetic_data=fake_df['sex']
)

0.6022531983960282

Так как у нас значение фиксированно, метрика плохая.

In [55]:
CorrelationSimilarity.compute(
    real_data=users_df[['bdate', 'counters.subscriptions']],
    synthetic_data=fake_df[['bdate', 'counters.subscriptions']],
    coefficient='Pearson'
)

0.9530822033795853

In [44]:
from sdmetrics.reports.single_table import QualityReport

report = QualityReport()
report.generate(real_data, fake_df, metadata.to_dict())

Generating report ...
(2/2) Evaluating Column Pair Trends: :   1%|          | 21/2278 [00:00<01:08, 32.79it/s]

/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


(2/2) Evaluating Column Pair Trends: :  23%|██▎       | 518/2278 [03:47<00:47, 37.24it/s]

/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


(2/2) Evaluating Column Pair Trends: :  23%|██▎       | 527/2278 [03:47<01:22, 21.22it/s]

...

(2/2) Evaluating Column Pair Trends: :  79%|███████▉  | 1805/2278 [06:19<00:24, 19.52it/s]

/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


(2/2) Evaluating Column Pair Trends: :  97%|█████████▋| 2203/2278 [07:10<00:29,  2.58it/s]

/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


(2/2) Evaluating Column Pair Trends: : 100%|██████████| 2278/2278 [07:23<00:00,  5.13it/s]

Overall Quality Score: 62.4%

Properties:
- Column Shapes: 52.42%
- Column Pair Trends: 72.38%


In [60]:
report.get_visualization(property_name='Column Shapes')



1.   Я генерирую дату только там, где есть год рождения. Надо ли генерить там где только месяц и день? модель вряд ли учитывает это при моделировании ответа
2.   Еще я использую ID, а надо настоящие значения признаков
3.   Имена не годятся, но и генерировать их невозможно, их под 4000 уникальных.
4.   Несколько условий можно задать!
5.   Надо следить за количеством категорий - много категорий\долго учится модель




# Разные ссылки на документацию

## Эксперименты с ганами

1. Amazon https://cseweb.ucsd.edu/~jmcauley/datasets.html#amazon_reviews - тут нет контекстов пользователя, только айтемов
2. Netflix Userbase dataset https://www.kaggle.com/datasets/arnavsmayan/netflix-userbase-dataset
3.



## Нужный код

0. репа с CTGAN https://github.com/sdv-dev/CTGAN
1. Реализация CGAN Lei Xu на торче - https://www.kaggle.com/code/gogo827jz/data-augmentation-ctgan
2. Еще одна реализация, попроще, с примером Adult Census Income Dataset и с Skin Detections  https://www.kaggle.com/code/karimkhaled/gans-for-tabular-data/notebook тут на этом датасете меньший клас стал еще меньше

3. ПАКЕТ КОТОРЫЙ ГЕНЕРИРУЕТ ДАННЫЕ - **SDV**   https://github.com/sdv-dev/SDV

## Всякая всячина

1. Парень не смог добиться лучшего качества на CTGAN потому что корреляционная матрица реальных и сгенерированных данных не совпадают  https://www.kaggle.com/competitions/lish-moa/discussion/193358

## Статейки


0. Conditional Tabular GAN https://proceedings.neurips.cc/paper/2019/file/254ed7d2de3b23ab10936522dd547b78-Paper.pdf
1. Wasserstain GAN - train GAN with Wasserstein distance https://arxiv.org/abs/1701.07875
2. PacGAC - allows cope with mode collapse - when data is generated with little diversity https://arxiv.org/abs/1712.04086

Mode collapse happens when the generator can only produce a single type of output or a small set of outputs. This may happen due to problems in training, such as the generator finding a type of data that is easily able to fool the discriminator and thus keeps generating that one type

1.  Autoencoder on Gaussian Mixture model https://www.researchgate.net/publication/348947965_Collaborative_Filtering_Based_on_a_Variational_Gaussian_Mixture_Model/download?_tp=eyJjb250ZXh0Ijp7ImZpcnN0UGFnZSI6Il9kaXJlY3QiLCJwYWdlIjoiX2RpcmVjdCJ9fQ
2. Variational AutoEncdoer https://jaan.io/what-is-variational-autoencoder-vae-tutorial/

1. Jensenn-Hannon divergence https://towardsdatascience.com/how-to-understand-and-use-jensen-shannon-divergence-b10e11b03fd6?source=read_next_recirc-----f3ddc8dff254----0---------------------258b436c_ea89_4e3e_8cc1_f2729dfb1ae2-------


In [ ]:
!wget https://files.grouplens.org/datasets/hetrec2011/hetrec2011-movielens-2k-v2.zip